In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [2]:
client = bigquery.Client(project='dataproject-2-413010')

## QUERY

query_df = """
SELECT p.*, 
v.nombre_conductor, v.marca_coche, v.color_coche, v.anio_fabricacion, 
v.tipo_combustible, v.tiene_calefaccion, v.puntos_carnet, v.cilindraje_motor, v.kilometraje, v.anio_registro_app,
c.nombre_cliente, c.apellido_cliente, c.edad_cliente, c.genero_cliente,
c.direccion_cliente, c.trabajo_cliente, c.fecha_registro,
FROM dataproject-2-413010.BBDD.CochePole_BD p
left join dataproject-2-413010.BBDD.tabla_vehiculos v on p.vehiculo_id = v.vehiculo_id
left join dataproject-2-413010.BBDD.tabla_clientes c on p.cliente_id = c.cliente_id
"""

df = client.query(query_df).to_dataframe()

df.dtypes

cliente_id                         Int64
rating                           float64
metodo_pago                       object
pago_viaje                       float64
viaje_id                           Int64
vehiculo_id                        Int64
latitud                          float64
longitud                         float64
latitud_final                    float64
longitud_final                   float64
timestamp            datetime64[ns, UTC]
nombre_conductor                  object
marca_coche                       object
color_coche                       object
anio_fabricacion                   Int64
tipo_combustible                  object
tiene_calefaccion                boolean
puntos_carnet                      Int64
cilindraje_motor                   Int64
kilometraje                        Int64
anio_registro_app                  Int64
nombre_cliente                    object
apellido_cliente                  object
edad_cliente                       Int64
genero_cliente  

In [3]:
pd.set_option('display.max_columns', None)

df.head()

,cliente_id,rating,metodo_pago,pago_viaje,viaje_id,vehiculo_id,latitud,longitud,latitud_final,longitud_final,timestamp,nombre_conductor,marca_coche,color_coche,anio_fabricacion,tipo_combustible,tiene_calefaccion,puntos_carnet,cilindraje_motor,kilometraje,anio_registro_app,nombre_cliente,apellido_cliente,edad_cliente,genero_cliente,direccion_cliente,trabajo_cliente,fecha_registro
0,83390,9.0,Efectivo,1.970438,2,387,39.47851,-0.40781,39.47117,-0.38696,2024-02-18 00:35:47.091378+00:00,Iker Guitart,Mazda,Amarillo claro,2011,Gasolina,True,10,1800,173715,2015,Simon,Neira,20,hombre,"Avenida Aureliano Abril 32 Piso 8 \nMadrid, 37209",Conductor de camiones pesados,2023-09-28
1,69340,9.0,Tarjeta,5.150495,28,934,39.44421,-0.37589,39.49002,-0.38533,2024-02-18 00:35:47.092015+00:00,Joaquina Carnero Naranjo,Chevrolet,Azul,2020,Gasolina,True,11,2200,46733,2021,Baudelio,Arnau,48,hombre,"Acceso de Clmaco Amaya 42\nZaragoza, 30956",Mecánico y reparador en electrónica,2023-04-15
2,51787,6.0,Tarjeta,3.036428,11,880,39.47888,-0.36348,39.45250,-0.35417,2024-02-18 00:35:47.092554+00:00,Elodia Borrego Garcés,Volkswagen,Negro,2005,Gasolina,True,7,1600,252463,2022,Ruben,Portero,32,hombre,Pasaje de Fidela Barcel 52 Puerta 7 \nAlicante...,Empleado de servicios de correos,2023-06-25
3,87226,7.0,Tarjeta,4.398410,10,82,39.46191,-0.33057,39.46558,-0.38146,2024-02-18 00:37:34.563616+00:00,Alma Segarra,MINI,Turquesa oscuro,2022,Gasolina,False,13,1400,14655,2023,Cristina,Cabrera,53,mujer,"Cuesta de Sosimo Valbuena 7\nCdiz, 23672",Conductor de vehículos accionados a pedal o a ...,2024-01-10
4,51315,7.0,Tarjeta,2.699485,25,124,39.49956,-0.40793,39.47662,-0.41833,2024-02-18 00:38:03.907670+00:00,Andrés Felipe Lopez Boada,Chevrolet,Marrón,2022,Gasolina,False,4,2200,17147,2022,Gaston,Pardo,36,hombre,"Camino Ana Sofa Abad 138\nBaleares, 25553",Secretario administrativo,2023-07-15


In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [6]:
columns_to_drop = ['pago_viaje', 'cliente_id', 'longitud', 'latitud', 'vehiculo_id', 'nombre_cliente', 'apellido_cliente', 'nombre_conductor', 'direccion_cliente']
X = df.drop(columns=columns_to_drop, axis=1)
y = df['pago_viaje']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Seleccionar las columnas numéricas, categóricas y booleanas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
boolean_features = X.select_dtypes(include=['boolean']).columns

# Crear transformadores para variables numéricas, categóricas y booleanas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

boolean_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Combinar transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ])

# Crear el modelo XGBoost
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Crear el pipeline con preprocesador y modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'model__n_estimators': [200, 300, 400],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [4, 5, 6],
    'model__subsample': [0.8, 0.9, 1.0],
    'model__colsample_bytree': [0.8, 0.9, 1.0],
}

# Configurar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Entrenar el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo del grid search
best_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.06109576840930155


In [7]:
numeric_columns_after_transform = best_model.named_steps['preprocessor'].transformers_[0][1]['scaler'].get_feature_names_out(numeric_features)

categorical_columns_after_transform = best_model.named_steps['preprocessor'].transformers_[1][1]['onehot'].get_feature_names_out(categorical_features)

all_columns_after_transform = list(numeric_columns_after_transform) + list(categorical_columns_after_transform) + list(boolean_features)

X_train_transformed = best_model.named_steps['preprocessor'].transform(X_train)
X_train_transformed_df = pd.DataFrame(X_train_transformed.toarray(), columns=all_columns_after_transform)

variance_of_used_columns = X_train_transformed_df.var()
print('Varianza de las variables utilizadas:\n', variance_of_used_columns)


Varianza de las variables utilizadas:
 rating                                                          1.003175
viaje_id                                                        1.003175
latitud_final                                                   1.003175
longitud_final                                                  1.003175
anio_fabricacion                                                1.003175
                                                                  ...   
trabajo_cliente_Vendedor de comidas al mostrador                0.006309
trabajo_cliente_Vendedor de quioscos y de puestos de mercado    0.003165
trabajo_cliente_Vendedor por teléfono                           0.006309
trabajo_cliente_Árbitro deportivo                               0.003165
tiene_calefaccion                                               0.248222
Length: 384, dtype: float64


In [8]:
variance_of_used_columns = X_train_transformed_df.var()

variance_of_used_columns_sorted = variance_of_used_columns.sort_values(ascending=True)

fig = px.bar(variance_of_used_columns_sorted, orientation='h', labels={'value': 'Varianza', 'index': 'Variables'},
             title='Varianza de las Variables Utilizadas (Ordenadas de Mayor a Menor)')

fig.show()

In [9]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rating', 'viaje_id', 'latitud_final', 'longitud_final',
       'anio_fabricacion', 'puntos_carnet', 'cilindraje_motor', 'kilometraje',
       'anio_registro_app', 'edad_cliente'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[(...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=400, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [10]:
import joblib

joblib.dump(best_model, 'modelo_final.joblib')

['modelo_final.joblib']

In [11]:
cliente_data = {
    'rating': [8.0],
    'metodo_pago': ['Tarjeta'],
    'viaje_id': [23],
    'latitud_final': [39.47947],
    'longitud_final': [-0.37594],
    'timestamp': ['2024-02-17 11:10:33.369962+00:00'],
    'marca_coche': ['BMW'],
    'color_coche': ['Marrón siena'],
    'anio_fabricacion': [2020],
    'tipo_combustible': ['Híbrido'],
    'tiene_calefaccion': [1],
    'puntos_carnet': [11],
    'cilindraje_motor': [2500],
    'kilometraje': [42353],
    'anio_registro_app': [2022],
    'edad_cliente': [48],
    'genero_cliente': ['mujer'],
    'trabajo_cliente': ['Parquetero y colocador de suelos'],
    'fecha_registro': ['2023-09-24']
}

cliente_df = pd.DataFrame(cliente_data)

prediction = best_model.predict(cliente_df)

print(f'Predicción de pago de viaje para el cliente: {prediction[0]}')


Predicción de pago de viaje para el cliente: 4.008509635925293
